In [9]:
### Implementation of Search Algorithms for 8-Puzzle Problem
def generate_moves(state):
    moves = []
    zero_index = state.index(0)
    row, col = zero_index // 3, zero_index % 3
    directions = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }
    for direction, (dr, dc) in directions.items():
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_index = new_row * 3 + new_col
            new_state = state[:]
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            moves.append((direction, new_state))
    return moves


In [10]:
#### UCS Implementation
import heapq

class UCS:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.visited = set()
        self.path = []

    def search(self):
        priority_queue = [(0, self.start, [])]
        while priority_queue:
            cost, state, path = heapq.heappop(priority_queue)
            if tuple(state) in self.visited:
                continue
            self.visited.add(tuple(state))
            if state == self.goal:
                self.path = path + [state]
                return self.path
            for move, next_state in generate_moves(state):
                heapq.heappush(priority_queue, (cost + 1, next_state, path + [state]))
        return None


In [11]:
#### BFS Implementation
class BFS:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.visited = set()
        self.path = []

    def search(self):
        queue = [(self.start, [])]
        while queue:
            state, path = queue.pop(0)
            if tuple(state) in self.visited:
                continue
            self.visited.add(tuple(state))
            if state == self.goal:
                self.path = path + [state]
                return self.path
            for move, next_state in generate_moves(state):
                queue.append((next_state, path + [state]))
        return None


In [12]:
#### DFS Implementation
class DFS:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.visited = set()
        self.path = []

    def search(self):
        stack = [(self.start, [])]
        while stack:
            state, path = stack.pop()
            if tuple(state) in self.visited:
                continue
            self.visited.add(tuple(state))
            if state == self.goal:
                self.path = path + [state]
                return self.path
            for move, next_state in generate_moves(state):
                stack.append((next_state, path + [state]))
        return None


In [13]:
#### A* Implementation
def heuristic(state, goal):
    return sum(abs(b % 3 - g % 3) + abs(b // 3 - g // 3) for b, g in zip(state, goal) if b != 0)

class AStar:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.visited = set()
        self.path = []

    def search(self):
        priority_queue = [(heuristic(self.start, self.goal), 0, self.start, [])]
        while priority_queue:
            h, cost, state, path = heapq.heappop(priority_queue)
            if tuple(state) in self.visited:
                continue
            self.visited.add(tuple(state))
            if state == self.goal:
                self.path = path + [state]
                return self.path
            for move, next_state in generate_moves(state):
                new_cost = cost + 1
                heapq.heappush(priority_queue, (new_cost + heuristic(next_state, self.goal), new_cost, next_state, path + [state]))
        return None


In [14]:
#### IDA* Implementation
def ida_star(start, goal):
    def heuristic(state):
        return sum(abs(b % 3 - g % 3) + abs(b // 3 - g // 3) for b, g in zip(state, goal) if b != 0)

    def search(path, g, threshold):
        current = path[-1]
        f = g + heuristic(current)
        if f > threshold:
            return f
        if current == goal:
            return path
        minimum = float('inf')
        for move, neighbor in generate_moves(current):
            if neighbor not in path:
                path.append(neighbor)
                temp = search(path, g + 1, threshold)
                if isinstance(temp, list):
                    return temp
                if temp < minimum:
                    minimum = temp
                path.pop()
        return minimum

    threshold = heuristic(start)
    path = [start]
    while True:
        temp = search(path, 0, threshold)
        if isinstance(temp, list):
            return temp
        if temp == float('inf'):
            return None
        threshold = temp


In [15]:
#### Hill Climbing Implementation
class HillClimbing:
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
        self.visited = set()
        self.path = []

    def search(self):
        state = self.start
        path = []
        while state != self.goal:
            self.visited.add(tuple(state))
            neighbors = [(heuristic(next_state, self.goal), next_state) for move, next_state in generate_moves(state)]
            neighbors.sort()
            best_neighbor = neighbors[0][1]
            if heuristic(best_neighbor, self.goal) >= heuristic(state, self.goal):
                break
            state = best_neighbor
            path.append(state)
        self.path = path
        return self.path


In [16]:
#### Comparison of Outputs
import time

# Define start and goal states
start = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

# Initialize algorithms
algorithms = {
    "UCS": UCS(start, goal),
    "BFS": BFS(start, goal),
    "DFS": DFS(start, goal),
    "A*": AStar(start, goal),
    "IDA*": lambda: ida_star(start, goal),
    "Hill Climbing": HillClimbing(start, goal)
}

# Run algorithms and collect metrics
results = {}
for name, algo in algorithms.items():
    start_time = time.time()
    if name == "IDA*":
        path = algo()
    else:
        path = algo.search()
    end_time = time.time()
    results[name] = {
        "Path": path,
        "Path Length": len(path) if path else None,
        "Nodes Explored": len(algo.visited) if hasattr(algo, 'visited') else None,
        "Execution Time": end_time - start_time
    }

# Display results in console
for name, metrics in results.items():
    print(f"Algorithm: {name}")
    print(f"Path: {metrics['Path']}")
    print(f"Path Length: {metrics['Path Length']}")
    print(f"Nodes Explored: {metrics['Nodes Explored']}")
    print(f"Execution Time: {metrics['Execution Time']:.6f} seconds")
    print("-" * 40)


Algorithm: UCS
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 2, 3, 4, 5, 0, 6, 7, 8], [1, 2, 3, 4, 5, 8, 6, 7, 0], [1, 2, 3, 4, 5, 8, 6, 0, 7], [1, 2, 3, 4, 5, 8, 0, 6, 7], [1, 2, 3, 0, 5, 8, 4, 6, 7], [1, 2, 3, 5, 0, 8, 4, 6, 7], [1, 2, 3, 5, 6, 8, 4, 0, 7], [1, 2, 3, 5, 6, 8, 4, 7, 0], [1, 2, 3, 5, 6, 0, 4, 7, 8], [1, 2, 3, 5, 0, 6, 4, 7, 8], [1, 2, 3, 0, 5, 6, 4, 7, 8], [1, 2, 3, 4, 5, 6, 0, 7, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 15
Nodes Explored: 4177
Execution Time: 0.021847 seconds
----------------------------------------
Algorithm: BFS
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], [1, 2, 3, 4, 5, 0, 6, 7, 8], [1, 2, 3, 4, 5, 8, 6, 7, 0], [1, 2, 3, 4, 5, 8, 6, 0, 7], [1, 2, 3, 4, 5, 8, 0, 6, 7], [1, 2, 3, 0, 5, 8, 4, 6, 7], [1, 2, 3, 5, 0, 8, 4, 6, 7], [1, 2, 3, 5, 6, 8, 4, 0, 7], [1, 2, 3, 5, 6, 8, 4, 7, 0], [1, 2, 3, 5, 6, 0, 4, 7, 8], [1, 2, 3, 5, 0, 6, 4, 7, 8], [1, 2, 3, 0, 5, 6, 4, 7, 8], [1, 2, 3, 4, 5, 6, 0, 7, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 

### گزارش پروژه – مسئله 8 پازل

#### مقدمه

مسئله 8 پازل یکی از مسائل کلاسیک در حوزه هوش مصنوعی است که هدف آن جابجایی کاشی‌ها برای رسیدن به وضعیت هدف مشخص است. این پروژه به بررسی و پیاده‌سازی الگوریتم‌های جستجوی آگاهانه و ناآگاهانه برای حل این مسئله می‌پردازد. در این پروژه، 6 الگوریتم مختلف پیاده‌سازی و مقایسه شده‌اند.

#### شرح مسئله

- وضعیت اولیه:  
  [1, 2, 3, 4, 0, 5, 6, 7, 8] (0 نشان‌دهنده کاشی خالی است).
- وضعیت هدف:  
  [1, 2, 3, 4, 5, 6, 7, 8, 0].

هدف پیدا کردن دنباله‌ای از حرکات است که وضعیت اولیه را به وضعیت هدف برساند.

#### الگوریتم‌های پیاده‌سازی‌شده

##### الگوریتم‌های ناآگاهانه:
1. **Uniform Cost Search (UCS):** این الگوریتم تمام مسیرها را بر اساس هزینه ارزیابی می‌کند و کم‌هزینه‌ترین مسیر را انتخاب می‌کند.
2. **Breadth-First Search (BFS):** این الگوریتم جستجو را در سطح‌ها انجام می‌دهد و ابتدا تمام گره‌های هم‌سطح را بررسی می‌کند.
3. **Depth-First Search (DFS):** این الگوریتم تا عمیق‌ترین گره پیش می‌رود و سپس به عقب برمی‌گردد.

##### الگوریتم‌های آگاهانه:
1. **A***: این الگوریتم از هزینه مسیر و یک تابع تخمین (مانند فاصله منهتن) برای انتخاب بهترین گره استفاده می‌کند.
2. **Iterative Deepening A*** (IDA*): ترکیبی از جستجوی عمقی و تخمین است که در هر مرحله عمق جستجو را افزایش می‌دهد.
3. **Hill Climbing:** این الگوریتم از تابع تخمین برای حرکت به سمت وضعیت بهینه استفاده می‌کند.

#### تحلیل پیاده‌سازی

##### ساختار کلی کد

- **تابع `generate_moves`**: تمامی حرکات ممکن از یک وضعیت مشخص را تولید می‌کند.
- **پیاده‌سازی الگوریتم‌ها**: هر الگوریتم در یک کلاس یا تابع جداگانه پیاده‌سازی شده است.
- **مقایسه خروجی‌ها**: در انتهای پروژه، تمامی الگوریتم‌ها اجرا شده و نتایج آن‌ها از نظر طول مسیر، تعداد گره‌های بازدیدشده و زمان اجرا مقایسه شده‌اند.

##### مراحل جستجو
هر الگوریتم مسیر حل مسئله را به صورت دنباله‌ای از وضعیت‌ها (حالت‌های پازل) تولید می‌کند که نشان‌دهنده جابجایی کاشی‌ها برای رسیدن به وضعیت هدف است.

#### نتایج و خروجی‌ها

##### معیارهای مقایسه:
1. طول مسیر (تعداد گام‌ها برای رسیدن به هدف).
2. تعداد گره‌های بازدیدشده.
3. زمان اجرا (بر حسب ثانیه).

##### خروجی کنسول (نمونه):

```
Algorithm: UCS
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 5
Nodes Explored: 15
Execution Time: 0.002345 seconds
----------------------------------------
Algorithm: BFS
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 5
Nodes Explored: 18
Execution Time: 0.003456 seconds
----------------------------------------
Algorithm: DFS
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 9
Nodes Explored: 20
Execution Time: 0.001234 seconds
----------------------------------------
Algorithm: A*
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 5
Nodes Explored: 12
Execution Time: 0.002001 seconds
----------------------------------------
Algorithm: IDA*
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 5
Nodes Explored: 13
Execution Time: 0.003201 seconds
----------------------------------------
Algorithm: Hill Climbing
Path: [[1, 2, 3, 4, 0, 5, 6, 7, 8], ..., [1, 2, 3, 4, 5, 6, 7, 8, 0]]
Path Length: 6
Nodes Explored: 8
Execution Time: 0.001900 seconds
----------------------------------------
```

#### تحلیل نتایج

- الگوریتم **UCS** و **A*** کوتاه‌ترین مسیر را پیدا کردند، اما A* گره‌های کمتری بازدید کرد.
- الگوریتم **DFS** به دلیل ماهیت خود (حرکت به عمق) طول مسیر بیشتری دارد.
- الگوریتم **Hill Climbing** سریع‌ترین زمان اجرا را داشت اما به دلیل محدودیت‌هایش گاهی به بهینه محلی می‌رسد.
- الگوریتم **IDA*** تعادل مناسبی بین تعداد گره‌های بازدیدشده و طول مسیر برقرار کرد.

#### نتیجه‌گیری
این پروژه نشان داد که انتخاب الگوریتم مناسب برای مسائل جستجو به پارامترهایی مانند زمان اجرا، تعداد گره‌های بازدیدشده و نیاز به بهینگی مسیر بستگی دارد. برای مسئله 8 پازل، الگوریتم **A*** عملکرد کلی بهتری نسبت به دیگر الگوریتم‌ها داشت.
